In [4]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow
from sklearn.metrics import mean_squared_error
import pandas as pd
from datetime import datetime
import pickle

In [6]:
MLFLOW_TRACKING_URI="http://localhost:5000"
client=MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)   

In [8]:
# client.list_experiments() # In older versions
client.search_experiments() # Ine the new version


[<Experiment: artifact_location='s3://mlflow/3', creation_time=1748273246641, experiment_id='3', last_update_time=1748273246641, lifecycle_stage='active', name='nyc-taxi', tags={}>,
 <Experiment: artifact_location='s3://mlflow/1', creation_time=1748029823417, experiment_id='1', last_update_time=1748029823417, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='s3://mlflow/0', creation_time=1748023896297, experiment_id='0', last_update_time=1748023896297, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
client.create_experiment(name="new-exp")

'4'

In [13]:
runs=client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 12",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=['metrics.rmse ASC']
)

for run in runs:
    print(f" run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f} ")

 run id: 85a887925d044d07a49b49e914a1f7f6, rmse: 9.2334 
 run id: 1bffb215154c422f98d22de98c374af4, rmse: 9.2334 
 run id: 70bf1274afd64e03bea9839437f53deb, rmse: 9.2334 
 run id: 6d0e6d81aa694a6ebe2fa12aa93f04ad, rmse: 11.1673 
 run id: b625bd5bd35f491ea899fb5a7e8f3e70, rmse: 11.1673 
 run id: a22b08bf410844389631a8ea4894d9ff, rmse: 11.1673 
 run id: fe0bd524f2bb47948f0b40db8f49896b, rmse: 11.1673 


In [14]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_id='70bf1274afd64e03bea9839437f53deb'
model_uri=f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="my-model")

Successfully registered model 'my-model'.
2025/05/27 06:34:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: my-model, version 1
Created version '1' of model 'my-model'.


<ModelVersion: aliases=[], creation_timestamp=1748327693160, current_stage='None', description='', last_updated_timestamp=1748327693160, name='my-model', run_id='70bf1274afd64e03bea9839437f53deb', run_link='', source='s3://mlflow/1/70bf1274afd64e03bea9839437f53deb/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [17]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748327693098, description='', last_updated_timestamp=1748327693160, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748327693160, current_stage='None', description='', last_updated_timestamp=1748327693160, name='my-model', run_id='70bf1274afd64e03bea9839437f53deb', run_link='', source='s3://mlflow/1/70bf1274afd64e03bea9839437f53deb/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='my-model', tags={}>]

In [20]:
model_name='my-model'
latest_versions=client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

/tmp/ipykernel_171/3677878745.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions=client.get_latest_versions(name=model_name)


version: 1, stage: None


In [24]:
model_version=1
new_stage='staging'
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_171/370265925.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748327693160, current_stage='Staging', description='', last_updated_timestamp=1748328236314, name='my-model', run_id='70bf1274afd64e03bea9839437f53deb', run_link='', source='s3://mlflow/1/70bf1274afd64e03bea9839437f53deb/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [25]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748327693098, description='', last_updated_timestamp=1748328236314, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748327693160, current_stage='Staging', description='', last_updated_timestamp=1748328236314, name='my-model', run_id='70bf1274afd64e03bea9839437f53deb', run_link='', source='s3://mlflow/1/70bf1274afd64e03bea9839437f53deb/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='my-model', tags={}>]

In [26]:
model_name = 'my-model'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging


/tmp/ipykernel_171/2407974716.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [27]:
date=datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1748327693160, current_stage='Staging', description='The model version 1 was transitioned to staging on 2025-05-27', last_updated_timestamp=1748328418109, name='my-model', run_id='70bf1274afd64e03bea9839437f53deb', run_link='', source='s3://mlflow/1/70bf1274afd64e03bea9839437f53deb/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [30]:
def read_data_frame(file):
    df = pd.read_parquet(file)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df,dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    
    return dv.transform(train_dicts)

def test_model(name,stage,x_test,y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(x_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}


In [31]:
df = read_data_frame('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')


In [ ]:
run_id = '70bf1274afd64e03bea9839437f53deb'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [ ]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
x_test = preprocess(df, dv[0])

In [ ]:
target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage="Production", x_test=x_test, y_test=y_test)